### Download data from Morse et al

https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE128033

There's a single ~500MB file with all counts and barcodes for analyzed samples. This should be.

In [69]:
import scanpy as sc
from os.path import join, exists
from os import listdir
import pandas as pd

d = '../data/morse'

### Metadata
- Currently metadata is nowwhere to be found.
- We use the LCA annotation metadata (atm, confirm labels).
- Replace h5ad path with the latest LCA h5ad generated

In [18]:
raw_annotated_lca_h5ad_path = ''
if exists(raw_annotated_lca_h5ad_path):
    lca = sc.read_h5ad(raw_annotated_lca_h5ad_path)
    laf = lca[lca.obs['dataset'].isin({'Pittsburgh_Lafyatis_2019Morse_10Xv1',
                                       'Pittsburgh_Lafyatis_2019Morse_10Xv2'}),:]
    laf.write('../data/morse/morse_lca_with_metadata.h5ad', compression='lzf')
    
    laf_mac = laf[laf.obs['original_celltype_ann'].str.contains('Macrophage'),:]
    laf_mac.write('../data/morse/morse_lca_with_metadata_macrophages.h5ad', compression='lzf')
laf_mac = sc.read_h5ad('../data/morse/morse_lca_with_metadata_macrophages.h5ad')

In [151]:
laf_mac_ad_by_sample = {sample: laf_mac[laf_mac.obs['sample'] == sample,:] for sample in set(laf_mac.obs['sample'])}

### Read raw counts

In [64]:
uniq_k = {"_".join(k.split("_")[:2]) for k in listdir(d) if f.endswith(".txt.gz")}
uniq_k

{'GSM3660641_SC14NOR',
 'GSM3660642_SC31NOR',
 'GSM3660643_SC31DNOR',
 'GSM3660644_SC45NOR',
 'GSM3660645_SC56NOR',
 'GSM3660646_SC59NOR',
 'GSM3660647_SC155NORLOW',
 'GSM3660648_SC156NORUP',
 'GSM3660649_SC228NORbal',
 'GSM3660650_SC249NORbal',
 'GSM3660651_SC87IPFLOW',
 'GSM3660652_SC88IPFUP',
 'GSM3660653_SC89IPFLOW',
 'GSM3660654_SC93IPFLOW',
 'GSM3660655_SC94IPFUP',
 'GSM3660656_SC95IPFLOW',
 'GSM3660657_SC153IPFLOW',
 'GSM3660658_SC154IPFUP'}

In [101]:
ad_by_k = {}

In [103]:
for code in uniq_k:
    has_lca_metadata = False
    for k_lca in laf_mac_ad_by_sample.keys():
        if k_lca in code:
            has_lca_metadata = True
    if not has_lca_metadata:
        print('reject (not LCA metadata)', code)
        continue
    print('reading', code)
    
    if code in ad_by_k:
        continue
    mtx_path = join(d, '%s_matrix.mtx.gz' % code)
    var_path = join(d, '%s_genes.tsv.gz' % code)
    obs_path = join(d, '%s_barcodes.tsv.gz' % code)

    if not exists(mtx_path):
        mtx_path = join(d, '%s_frozen_matrix.mtx.gz' % code)
        var_path = join(d, '%s_frozen_genes.tsv.gz' % code)
        obs_path = join(d, '%s_frozen_barcodes.tsv.gz' % code)
    if not exists(mtx_path):
        mtx_path = join(d, '%s_fresh_matrix.mtx.gz' % code)
        var_path = join(d, '%s_fresh_genes.tsv.gz' % code)
        obs_path = join(d, '%s_fresh_barcodes.tsv.gz' % code)
        
    ad = sc.read_mtx(mtx_path, dtype='float32').transpose()
    obs = pd.read_csv(obs_path, header=None, sep='\t')
    var = pd.read_csv(var_path, header=None, sep='\t')
    
    var.columns = ['ensembl', 'symbol'] if len(var.columns) == 2 else ['ensembl', 'symbol', 'readout']
    var.index = var.ensembl
    ad.var = var
    obs.columns = ['barcode']
    ad.obs.index = obs['barcode']
    
    ad.obs['GSM'] = code.split('_')[0]
    ad.obs['sample.id'] = code.split('_')[1]
    ad_by_k[code] = ad

GSM3660644_SC45NOR
reading GSM3660644_SC45NOR
GSM3660652_SC88IPFUP
reject (not LCA metadata) GSM3660652_SC88IPFUP
GSM3660654_SC93IPFLOW
reject (not LCA metadata) GSM3660654_SC93IPFLOW
GSM3660651_SC87IPFLOW
reject (not LCA metadata) GSM3660651_SC87IPFLOW
GSM3660657_SC153IPFLOW
reject (not LCA metadata) GSM3660657_SC153IPFLOW
GSM3660656_SC95IPFLOW
reject (not LCA metadata) GSM3660656_SC95IPFLOW
GSM3660658_SC154IPFUP
reject (not LCA metadata) GSM3660658_SC154IPFUP
GSM3660647_SC155NORLOW
reading GSM3660647_SC155NORLOW
GSM3660649_SC228NORbal
reject (not LCA metadata) GSM3660649_SC228NORbal
GSM3660646_SC59NOR
reading GSM3660646_SC59NOR
GSM3660650_SC249NORbal
reject (not LCA metadata) GSM3660650_SC249NORbal
GSM3660641_SC14NOR
reading GSM3660641_SC14NOR
GSM3660655_SC94IPFUP
reject (not LCA metadata) GSM3660655_SC94IPFUP
GSM3660645_SC56NOR
reading GSM3660645_SC56NOR
GSM3660648_SC156NORUP
reading GSM3660648_SC156NORUP
GSM3660642_SC31NOR
reading GSM3660642_SC31NOR
GSM3660643_SC31DNOR
reading GSM3

### Include information from LCA in each case and then save

In [152]:
ad_macrophages_by_code = {}
for code in ad_by_k:
    ad = ad_by_k[code]
    
    lca_k_sel = None
    for k_lca in laf_mac_ad_by_sample.keys():
        if k_lca in code:
            lca_k_sel = k_lca

    print(code, metadata.shape)
    
    metadata = pd.DataFrame(laf_mac_ad_by_sample[lca_k_sel].obs.copy())
    metadata['10x.barcode'] = set(metadata.index.str.split('_').str[0] + '-1')
    metadata.index = metadata['10x.barcode']
    sel = ad[ad.obs.index.isin(metadata.index),:]
    for c in metadata.reindex(sel.obs.index):
        sel.obs[c] = metadata[c]



Trying to set attribute `.obs` of view, copying.


GSM3660644_SC45NOR (2074, 63)
GSM3660647_SC155NORLOW (2074, 63)


Trying to set attribute `.obs` of view, copying.


GSM3660646_SC59NOR (1808, 63)


Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


...

GSM3660648_SC156NORUP (1140, 63)


Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


GSM3660642_SC31NOR (2235, 63)


Trying to set attribute `.obs` of view, copying.


GSM3660643_SC31DNOR (458, 63)


In [175]:

# concatenate
ad_macrophages_by_code.keys()
ad_mac = None
for k in ad_macrophages_by_code:
    print(k)
    next_ad = ad_macrophages_by_code[k].copy()
    next_ad.obs.index += '-' + k
    print(list(next_ad.obs.index[:5]))
    if ad_mac is not None:
        ad_mac = ad_mac.concatenate(next_ad)
        ad_mac.obs.index = ad_mac.obs.index.str[:-2]
    else:
        ad_mac = next_ad
ad_mac.var.index = list(ad_mac.var.index)

SC45
['AAACCTGGTCATGCCG-1-SC45', 'AAACCTGGTGGTGTAG-1-SC45', 'AAACCTGGTTGTACAC-1-SC45', 'AAACCTGTCACAACGT-1-SC45', 'AAACGGGCACAGCGTC-1-SC45']
SC155
['AAACCTGAGACAAGCC-1-SC155', 'AAACCTGAGTTGTCGT-1-SC155', 'AAACCTGGTAGCAAAT-1-SC155', 'AAACCTGGTGCACGAA-1-SC155', 'AAACCTGGTTAGAACA-1-SC155']
SC59
['AAACCTGAGATGGGTC-1-SC59', 'AAACCTGCACTGCCAG-1-SC59', 'AAACCTGTCTACCAGA-1-SC59', 'AAACGGGAGAGGTTAT-1-SC59', 'AAACGGGAGCTCTCGG-1-SC59']
SC14
['AAAGATCTTTACTC-1-SC14', 'AAATCTGAGTCATG-1-SC14', 'AACAAACTCCAACA-1-SC14', 'AACAAACTGAACCT-1-SC14', 'AACAATACTGAGAA-1-SC14']
SC56
['AAACCTGCACGGTAGA-1-SC56', 'AAACCTGGTTCCGGCA-1-SC56', 'AAACCTGGTTTAAGCC-1-SC56', 'AAACCTGTCATTGCGA-1-SC56', 'AAACGGGAGAGTACAT-1-SC56']
SC156
['AAACCTGAGCACGCCT-1-SC156', 'AAACCTGGTCATGCAT-1-SC156', 'AAACCTGGTTCCGGCA-1-SC156', 'AAACCTGTCGTAGATC-1-SC156', 'AAACCTGTCTAACTCT-1-SC156']
SC31
['AAACATTGACCCAA-1-SC31', 'AAACCGTGTGACAC-1-SC31', 'AAACGCTGAACCGT-1-SC31', 'AAAGACGAGGATTC-1-SC31', 'AAAGCAGATTGAGC-1-SC31']


In [189]:
ad_mac.var

,ensembl,symbol
ENSG00000243485,ENSG00000243485,RP11-34P13.3
ENSG00000237613,ENSG00000237613,FAM138A
ENSG00000186092,ENSG00000186092,OR4F5
ENSG00000238009,ENSG00000238009,RP11-34P13.7
ENSG00000239945,ENSG00000239945,RP11-34P13.8
...,...,...
ENSG00000277856,ENSG00000277856,AC233755.2
ENSG00000275063,ENSG00000275063,AC233755.1
ENSG00000271254,ENSG00000271254,AC240274.1
ENSG00000277475,ENSG00000277475,AC213203.1


In [191]:
ad_mac.write('../data/morse/morse_input_scvi.h5ad', compression='lzf')